## WARNING! for this file you need classla. you might get conflicts with tensorflow so i suggest you use a separate python environment for this one

in this notebook we are tokenizin input text (text from promet.si up to one hour before the reading of traffic news) and the output (the traffic news)

so if we have traffic news from 1.1.2024 11:30, we will have as input texts all the rows of the excel file (probably web scraped) that are from 1.1.2024 10:30 to 11:30

In [2]:
import classla
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
classla.download('sl')

2025-05-22 22:06:20 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2025-05-22 22:06:23 INFO: File exists: /home/nina/classla_resources/sl/pos/standard.pt.
2025-05-22 22:06:23 INFO: File exists: /home/nina/classla_resources/sl/lemma/standard.pt.
2025-05-22 22:06:23 INFO: File exists: /home/nina/classla_resources/sl/depparse/standard.pt.
2025-05-22 22:06:23 INFO: File exists: /home/nina/classla_resources/sl/ner/standard.pt.
2025-05-22 22:06:23 INFO: File exists: /home/nina/classla_resources/sl/pretrain/standard.pt.
2025-05-22 22:06:23 INFO: Finished downloading models and saved to /home/nina/classla_resources.


In [3]:
nlp = classla.Pipeline('sl', processors='tokenize,pos,lemma')

2025-05-23 16:44:02 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |

2025-05-23 16:44:02 INFO: Use device: cpu
2025-05-23 16:44:02 INFO: Loading: tokenize
2025-05-23 16:44:02 INFO: Loading: pos


2025-05-23 16:44:14 INFO: Loading: lemma
2025-05-23 16:44:30 INFO: Done loading processors!


In [4]:
test_output = 'Prometne informacije       01. 01. 2022  \t   11.30          2. program \n\nPodatki o prometu.\n\nNa gorenjski avtocesti proti Ljubljani je zaradi gorečega vozila zaprt vozni pas med priključkoma Brezje in Podtabor.   \n\nDanes do 21-ih velja prepoved prometa tovornih vozil, težjih od 7 ton in pol.\n'
test_input = 'Vreme Ponekod v osrednji Sloveniji megla v pasovih zmanjšuje vidljivost. Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes do 22. ure; - v nedeljo, 2. januarja, od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.'
# lets remove first row of student report when comparing texts
test_output = test_output.split('\n', 1)[1]
doc_out = nlp(test_output)
doc_in = nlp(test_input)
doc_out

[
  [
    [
      {
        "id": 1,
        "text": "Podatki",
        "lemma": "podatek",
        "upos": "NOUN",
        "xpos": "Ncmpn",
        "feats": "Case=Nom|Gender=Masc|Number=Plur"
      },
      {
        "id": 2,
        "text": "o",
        "lemma": "o",
        "upos": "ADP",
        "xpos": "Sl",
        "feats": "Case=Loc"
      },
      {
        "id": 3,
        "text": "prometu",
        "lemma": "promet",
        "upos": "NOUN",
        "xpos": "Ncmsl",
        "feats": "Case=Loc|Gender=Masc|Number=Sing",
        "misc": "SpaceAfter=No"
      },
      {
        "id": 4,
        "text": ".",
        "lemma": ".",
        "upos": "PUNCT",
        "xpos": "Z"
      }
    ],
    "# newpar id = 1\n# sent_id = 1.1\n# text = Podatki o prometu.\n"
  ],
  [
    [
      {
        "id": 1,
        "text": "Na",
        "lemma": "na",
        "upos": "ADP",
        "xpos": "Sl",
        "feats": "Case=Loc"
      },
      {
        "id": 2,
        "text": "gorenjski",
       

In [5]:
def lemma_string(doc):
  list_of_lemmas = [word.lemma for t in doc.iter_tokens() for word in t.words]
  odstrani_locila = [x for x in list_of_lemmas if x not in [',','.', ':', '-', ';']]
  list_to_string = ' '.join(odstrani_locila)
  return list_to_string

lemmas_out = lemma_string(doc_out)
print(lemmas_out)
lemmas_in = lemma_string(doc_in)
print(lemmas_in)

podatek o promet na gorenjski avtocesta proti Ljubljana biti zaradi goreč vozilo zaprt vozen pas med priključek Brezje in Podtabor danes do 21-ih veljati prepoved promet tovoren vozilo težek od 7 tona in pol
vreme ponekod v osrednji Slovenija megla v pas zmanjševati vidljivost omejitev za tovoren vozilo po Slovenija veljati med praznik omejitev za tovoren vozilo z velik dovoljen masa nad 7,5 tona danes do 22. ura v nedelja 2. januar od 8. do 22. ura od 30. december biti v veljava sprememba omejitev za tovoren vozilo nad 7,5 tona več


In [6]:
def pairwise_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

print(pairwise_similarity(lemmas_out, lemmas_in))

0.2124380819594513


In [ ]:
nlp_ner = classla.Pipeline('sl', processors='tokenize,ner,pos,lemma')

2025-05-23 16:46:05 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| ner       | standard |

2025-05-23 16:46:05 INFO: Use device: cpu
2025-05-23 16:46:05 INFO: Loading: tokenize
2025-05-23 16:46:05 INFO: Loading: pos
2025-05-23 16:46:17 INFO: Loading: lemma
